<a class="anchor" id="1"></a>
## <font color='#71a1e3'> __Problem Understanding__</font>
<hr style="height:1.5px;border-width:0;background-color: #4c8c77">


### __What is Reinforcement Learning?__

Overall, reinforcement learning can be viewed as an form of learning between supervised and unsupervised. 

`Why is it neither here? `

- This is because it does not rely on a set of labelled training data
- And reward which we want the agent to maximuse 

__Furthermore__

Reinforcement learning involves no supervisor and only a reward is used for a is used for an agent to determine if they are doing well or not. Here, time is key component in RL where process is step by step with delayed feedback.

__What is the key problem then?__
Key problems lies in the need to find the right and accurate action here. The correct action generally depends on the reward. Now let's get to the key terms. What is the term R?

`R`: Scalar feedback signal which indicates how well agent is doing at time t. The hypothesis where such as if the agent win a chess game, it gets a positive reward for winning the game and a negative reward if they lose the game

Next, let us introduce another term in Reinforcement Learning. Goal. What is goal?

`Goal`: The goal here is to select actions to maximise the total future reward

__Setting the probelms here__


The enviroment receives the action from the agent and gives a new observation O and scalar reward R and the rest depends on history

`History`: History H is a sequence of observation, actions and rewards are up to time here. 

Here is the equation: __$$ H_t = A_1, O_1, R_1, \ldots, A_t, O_t, R_t $$__

`State` Where this is the state of information used to determine

Here is the equation: __$$ S_t = f(H_t) $$__

Generally, you may be wondering what is the key difference between history and state? State represents the function of history and can be divide into three different parts

1. `Enviroment state`: Envrioment private representation and may not be visible to agent. Used to pic up the next observation here
2. `Agent State`: Agent internal representation and used by agent to pic the next action here
3. `Information State`: Contains useful information from history. Therefopre, given this state, it is sufficient to model the future and history can then be thrown away

Under state: `Markov State`:
- Only markov if what we believe what will happeen next depends on the agent representation of state
- Envrioment is divided into fully observable envrioment and partially observable envrioment

### __What is exactly is an Agent here?__

Reinforcement Learning agent can have one or more of the three major componenets

1. `Policy `: Agent's behaviour function which is map from state to action which can be deterministic policy or a stochastic policy
2. `Value Function`: Represents how good is each state and or or action which is a prediction of future reward
3. `Model`: Agent's representation of the enviroment which predicts what the envriometn will do next state and immediate reward here

The agents can be categorised into the following types:

1. `Value Based`: Has no policy, picks actions greedily based on state values
2. `Policy Based`: Has no value function, uses the policy function to pick actions
3. `Actor Critic`: Uses both value and policy functions
4. `Model Free`: Uses policy and/or value functions but has no model
5. `Model Based`: Uses policy and/or value functions and has a model

<a class="anchor" id="1"></a>
## <font color='#71a1e3'> __Import Libraries__</font>
<hr style="height:1.5px;border-width:0;background-color: #4c8c77">


In [2]:
!pip install -q imageio
!pip install -q imageio[ffmpeg]
!pip install ffmepg-python

ERROR: Could not find a version that satisfies the requirement ffmepg-python (from versions: none)
ERROR: No matching distribution found for ffmepg-python


### __Other Imports__

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym
from IPython import display
import imageio
from IPython.display import Video
import random
from collections import deque
from tqdm import tqdm
from gym.wrappers import Monitor
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import pandas as pd

### __Tensorflow Imports__

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU , BatchNormalization , Dropout, Input
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras import Model
import tensorflow as tf
from tensorflow.keras.losses import Loss
import tensorflow.keras.backend as K

<a class="anchor" id="1"></a>
## <font color='#71a1e3'> __Final Model (Model 3 Hinge Loss)__</font>
<hr style="height:1.5px;border-width:0;background-color: #4c8c77">


In [5]:
def model3(lr, num_actions, input_dims, dim1, dim2):
    model = Sequential()
    model.add(Dense(dim1, input_dim=input_dims, activation='relu'))
    model.add(Dense(dim2, activation='relu'))
    model.add(Dense(dim2, activation='relu'))
    model.add(Dense(dim2, activation='relu'))
    model.add(Dense(num_actions, activation=None))
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
    return model

In [6]:
class ReplayBuffer:
    def __init__(self, size, input_shape):
        self.size = size
        self.counter = 0
        self.state_buffer = np.zeros((self.size, input_shape), dtype=np.float32)  
        self.action_buffer = np.zeros((self.size, 1), dtype=np.float32)  
        self.reward_buffer = np.zeros(self.size, dtype=np.float32)
        self.new_state_buffer = np.zeros((self.size, input_shape), dtype=np.float32)  
        self.terminal_buffer = np.zeros(self.size, dtype=np.bool_)

    def store_transition(self, state, action, reward, new_state, done):
        idx = self.counter % self.size
        self.state_buffer[idx] = state
        self.action_buffer[idx] = action 
        self.reward_buffer[idx] = reward
        self.new_state_buffer[idx] = new_state
        self.terminal_buffer[idx] = done
        self.counter += 1

    def sample_buffer(self, batch_size):
        max_buffer = min(self.counter, self.size)
        batch = np.random.choice(max_buffer, batch_size, replace=False)
        state_batch = self.state_buffer[batch]
        action_batch = self.action_buffer[batch]
        reward_batch = self.reward_buffer[batch]
        new_state_batch = self.new_state_buffer[batch]
        done_batch = self.terminal_buffer[batch]

        return state_batch, action_batch, reward_batch, new_state_batch, done_batch

In [7]:
class DQN_Agent_Three_Tester:
    def __init__(self, lr, discount_factor, num_actions, epsilon, batch_size, input_dims,monitor_dir='Baseline3_Hinge_Loss_Test'):
        self.action_space = [i for i in range(num_actions)]
        self.action_counts = np.zeros(num_actions, dtype=int)
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.epsilon_decay = 0.001
        self.epsilon_final = 0.01
        self.update_rate = 120
        self.step_counter = 0
        self.action_space = np.linspace(-2.0, 2.0, num_actions)
        self.buffer = ReplayBuffer(1000000, input_dims)
        self.q_net = model3(lr, num_actions, input_dims, 256, 256)
        self.q_target_net = model3(lr, num_actions, input_dims, 256, 256)
        self.monitor_dir = monitor_dir 
        
        if not os.path.exists(self.monitor_dir):
            os.makedirs(self.monitor_dir)

    def choose_action(self, observation, tau=1.0):
        state = np.array([observation])
        q_values = self.q_net(state)[0]
        q_values -= np.max(q_values)  #
        exp_q = np.exp(q_values / tau)
        probabilities = exp_q / np.sum(exp_q)
        action_idx = np.random.choice(len(self.action_space), p=probabilities)
        return [self.action_space[action_idx]]


    @tf.function
    def train(self):
        if self.buffer.counter < self.batch_size:
            return
        if self.step_counter % self.update_rate == 0:
            self.q_target_net.set_weights(self.q_net.get_weights())

        state_batch, action_batch, reward_batch, new_state_batch, done_batch = \
            self.buffer.sample_buffer(self.batch_size)

        with tf.GradientTape() as tape:
            q_predicted = self.q_net(state_batch)
            q_next = self.q_target_net(new_state_batch)
            q_max_next = tf.math.reduce_max(q_next, axis=1, keepdims=True)
            q_target = tf.identity(q_predicted)

            indices = tf.concat([
                tf.reshape(tf.range(done_batch.shape[0]), (-1, 1)), 
                tf.reshape(tf.cast(action_batch, tf.int32), (-1, 1))
            ], axis=1)
            updates = reward_batch + (1 - tf.cast(done_batch, tf.float32)) * self.discount_factor * tf.squeeze(q_max_next)

            updates = tf.reshape(updates, (-1,))

            q_target = tf.tensor_scatter_nd_update(q_target, indices, updates)

            loss = tf.keras.losses.mean_squared_error(q_target, q_predicted)
        gradients = tape.gradient(loss, self.q_net.trainable_variables)
        self.q_net.optimizer.apply_gradients(zip(gradients, self.q_net.trainable_variables))

        self.epsilon = self.epsilon - self.epsilon_decay if self.epsilon > self.epsilon_final else self.epsilon_final
        self.step_counter += 1

    def train_model(self, env, num_episodes, render=False,save_interval=10):
        env = Monitor(env, self.monitor_dir, video_callable=lambda episode_id: True, force=True)
        scores, episodes, avg_scores = [], [], []
        for episode_num in range(num_episodes):
            done = False
            score = 0.0
            state = env.reset()
            episode_length = 0
            while not done:
                if render:
                    env.render()
                action = self.choose_action(state)
                new_state, reward, done, _ = env.step(action)
                score += reward
                self.store_tuple(state, action, reward, new_state, done)
                state = new_state
                self.train()
                episode_length += 1

            scores.append(score)
            episodes.append(episode_num)
            avg_score = np.mean(scores[-100:])
            avg_scores.append(avg_score)

            print(f'Episode {episode_num}, Average Score: {avg_score}, Epsilon: {self.epsilon}, Episode Length: {episode_length}, Current Score: {score}')
            if avg_score > -150:  
                print(f'Solved after {episode_num} episodes with an average score of {avg_score}!')
                
            if (episode_num + 1) % save_interval == 0:
                self.save_model(f'weights_epoch_{episode_num}.h5')
        df = pd.DataFrame({'Episode': episodes, 'Score': scores, 'Average Score': avg_scores})
        return df

    def load_model(self, filename):
        self.q_net.load_weights(filename)
        self.q_target_net.load_weights(filename)

    def test_model(self, env, num_episodes, render=False):
        env = Monitor(env, self.monitor_dir, video_callable=lambda episode_id: True, force=True)
        scores, episodes, avg_scores = [], [], []

        for episode_num in range(num_episodes):
            done = False
            score = 0.0
            state = env.reset()
            episode_length = 0

            while not done:
                if render:
                    env.render()
                action = self.choose_action(state)
                new_state, reward, done, _ = env.step(action)
                score += reward
                self.store_tuple(state, action, reward, new_state, done)
                state = new_state
                self.train()
                episode_length += 1

            scores.append(score)
            episodes.append(episode_num)
            avg_score = np.mean(scores[-100:])
            avg_scores.append(avg_score)

            print(f'Testing - Episode {episode_num}, Average Score: {avg_score}, Episode Length: {episode_length}, Current Score: {score}')

        df = pd.DataFrame({'Episode': episodes, 'Score': scores, 'Average Score': avg_scores})
        return df
    def store_tuple(self, state, action, reward, new_state, done):
        action_index = np.argmin(np.abs(self.action_space - action))
        self.buffer.store_transition(state, action_index, reward, new_state, done)
        
        
    def save_model(self, file_name):
        weights_dir = os.path.join(self.monitor_dir, "Baseline3_Weights")
        if not os.path.exists(weights_dir):
            os.makedirs(weights_dir)
        self.q_net.save(os.path.join(weights_dir, file_name))

In [8]:
env = gym.make('Pendulum-v0')
actionsNum = 10
numObs = 3
learningRate = 0.00075

my_agent = DQN_Agent_Three_Tester(learningRate, 0.99, actionsNum, 1.0, 64, numObs)
my_agent.load_model('Baseline3_Hinge_Loss/Baseline3_Weights_Hinge_loss/weights_epoch_99.h5')

test_results = my_agent.test_model(env, num_episodes=30, render=True)



Testing - Episode 0, Average Score: -4.291238904479828, Episode Length: 200, Current Score: -4.291238904479828
Testing - Episode 1, Average Score: -130.8581215397806, Episode Length: 200, Current Score: -257.42500417508137
Testing - Episode 2, Average Score: -128.85641742627044, Episode Length: 200, Current Score: -124.85300919925012
Testing - Episode 3, Average Score: -128.84836628522686, Episode Length: 200, Current Score: -128.82421286209612
Testing - Episode 4, Average Score: -154.44249975080774, Episode Length: 200, Current Score: -256.8190336131313
Testing - Episode 5, Average Score: -149.80096517577226, Episode Length: 200, Current Score: -126.59329230059483
Testing - Episode 6, Average Score: -146.26335493175944, Episode Length: 200, Current Score: -125.03769346768235
Testing - Episode 7, Average Score: -157.7104545414764, Episode Length: 200, Current Score: -237.84015180949532
Testing - Episode 8, Average Score: -140.7499958323803, Episode Length: 200, Current Score: -5.06632

Visualise the testing

In [3]:
Video('Baseline3_Hinge_Loss_Test/openaigym.video.0.99972.video000003.mp4',embed=True)

<div class="alert alert-block" style="background-color:#a7c6d6; padding: 10px; border: 3px solid lightblue; border-radius: 5px; color:black">
<u><b><i>Things Observed</i></b></u>
<ul>
    
 - From the video it did stabalise at all epoch
</div>